In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/covid19-global-forecasting-week-2/submission.csv
/kaggle/input/covid19-global-forecasting-week-2/test.csv
/kaggle/input/covid19-global-forecasting-week-2/train.csv


In [2]:
train=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/train.csv')
test=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/test.csv')
submission=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/submission.csv')

In [3]:
print(train.shape)
print(test.shape)
print(submission.shape)

(20580, 6)
(12642, 4)
(12642, 3)


In [4]:
train.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [5]:
test.head()

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-03-19
1,2,NaN,Afghanistan,2020-03-20
2,3,NaN,Afghanistan,2020-03-21
3,4,NaN,Afghanistan,2020-03-22
4,5,NaN,Afghanistan,2020-03-23


In [6]:
submission.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,1


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20580 entries, 0 to 20579
Data columns (total 6 columns):
Id                20580 non-null int64
Province_State    8750 non-null object
Country_Region    20580 non-null object
Date              20580 non-null object
ConfirmedCases    20580 non-null float64
Fatalities        20580 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 964.8+ KB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12642 entries, 0 to 12641
Data columns (total 4 columns):
ForecastId        12642 non-null int64
Province_State    5375 non-null object
Country_Region    12642 non-null object
Date              12642 non-null object
dtypes: int64(1), object(3)
memory usage: 395.2+ KB


In [9]:
train=train[train.Date>'2020-02-17']
X_train=train.drop(columns=['Id','ConfirmedCases','Fatalities'])
y_train_cc=train.ConfirmedCases
y_train_ft=train.Fatalities

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
impute=SimpleImputer(strategy='most_frequent')
X_train_1=impute.fit_transform(X_train)
X_train_2=OneHotEncoder(handle_unknown='ignore').fit_transform(X_train_1)

In [11]:
X_test=test.drop(columns=['ForecastId'])
X_test_1=impute.fit_transform(X_test)
X_test_2=OneHotEncoder(handle_unknown='ignore').fit_transform(X_test_1)

In [12]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
model_cc=RandomForestRegressor()
model_cc.fit(X_train_2, y_train_cc)
model_cc.score(X_train_2, y_train_cc)

0.9274853571093574

In [13]:
y_pred_cc=model_cc.predict(X_test_2)

In [14]:
model_ft=RandomForestRegressor()
model_ft.fit(X_train_2,y_train_ft)
model_ft.score(X_train_2, y_train_ft)

0.8941102594126924

In [15]:
y_pred_ft=model_ft.predict(X_test_2)
y_pred_ft

array([0.  , 0.  , 0.  , ..., 0.54, 0.63, 0.6 ])

In [16]:
submission.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,1


In [17]:
result=pd.DataFrame({'ForecastId':submission.ForecastId, 'ConfirmedCases':y_pred_cc, 'Fatalities':y_pred_ft})
result.to_csv('/kaggle/working/submission.csv', index=False)
data=pd.read_csv('/kaggle/working/submission.csv')
data.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,0.32,0.0
1,2,0.30,0.0
2,3,0.39,0.0
3,4,0.26,0.0
4,5,0.31,0.0
